In [1]:
import numpy as np                          # Basic Array and Numaric Operation
import os                                   # use to access the files 
import tarfile                              # use to extract dataset from zip files
import sys
import zipfile                              # to extract zip file


import torch                                # Provides basic tensor operation and nn operation
import torchvision                          # Provides facilities to access image dataset


import my_utils.loadDataset as dl           # create dataloader for selected dataset
import my_utils.loadModel as lm             # facilitate loading and manipulating models
import my_utils.trainModel as tm            # Facilitate training of the model
import my_utils.initialize_pruning as ip    # Initialize and provide basic parmeter require for pruning
import my_utils.facilitate_pruning as fp    # Compute Pruning Value and many things

Linear(in_features=4096, out_features=1000, bias=True)
Feature List  = [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M']
prune list    = [0, 2, 5, 7, 10, 12, 14, 17, 19, 21, 24, 26, 28] 
vgg net block = [2, 2, 3, 3, 3] 
Prune Count   = [1, 1, 2, 2, 4, 4, 9, 13, 13, 13]


### Prepare dataloader and set image properties. Check for cuda device and load model

In [2]:
# set the locationn of the dataset and trai and test data folder name
dl.setFolderLocation(datasets       ='/home/pragnesh/Dataset/',
                     selectedDataset='IntelIC/',
                     train          ='train',
                     test           ='test')
# set the imge properties
dl.setImageSize(224)
dl.setBatchSize = 2
dataLoader = dl.dataLoader()

#load the saved model if have any
load_path = "/home/pragnesh/Model/Model/VGG_IntelIC_v2"
device1 = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
newModel = torch.load(load_path, map_location=torch.device('cpu'))

#if dont have any saved trained model download pretrained model for tranfer learning
# newmodel = lm.load_model(model_name='vgg16',number_of_class=6,pretrainval=False,
#                         freeze_feature=False,device_l=device1)

#print(newModel)

### Here we have 3 set of parameter.One for data loader, second for model and third are hyperparameter
### Train the model

In [ ]:
logFile = '/home/pragnesh/Dataset/Intel_Image_Classifacation_v2/Logs/ConvModelv2.log'
#tm.device = torch.device('cpu')
tm.fit_one_cycle(#set locations of the dataset, train and test data
                 dataloaders=dataLoader,trainDir=dl.trainDir,testDir=dl.testDir,
                 # Selecat a variant of VGGNet
                 ModelName='vgg16',model=newModel,device_l=device1,
                 # Set all the Hyper-Parameter for training
                 epochs=1, max_lr=0.01, weight_decay=0, L1=0, grad_clip=.1, logFile=logFile)

#Save the  trained model 
SavePath = '/home/pragnesh/Model/vgg16-v2'
torch.save(newModel, SavePath)

### Pruning Initialization

In [3]:
blockList   = ip.createBlockList(newModel)              #ip.getBlockList('vgg16')
featureList = ip.createFeatureList(newModel)
convIdx     = ip.findConvIndex(newModel)
module      = ip.getPruneModule(newModel)
print(f"Block List   = {blockList}\n"
      f'Feature List = {featureList}\n' 
      f"Conv Index   = {convIdx}\n"
     )
for i in range(len(module)):
    print(f"{i+1} :{module[i]}")

Block List   = [2, 2, 3, 3, 3]
Feature List = [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M']
Conv Index   = [0, 2, 5, 7, 10, 12, 14, 17, 19, 21, 24, 26, 28]

1 :Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
2 :Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
3 :Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
4 :Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
5 :Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
6 :Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
7 :Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
8 :Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
9 :Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
10 :Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
11 :Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
12 :Conv2d(512, 512, kernel_size=(3, 3

### Pruning Process Starts From Here
##### compute score for pruning criteria for candidate convolution pruning layer

In [4]:
candidateConvLayer = []
st =0
en = 4
for i in range(st,en):
    candidateConvLayer.append(fp.compute_distance_score(module[i]._parameters['weight'],threshold=1))
print("\n\n\nHere is the :",len(candidateConvLayer))


Shape of the tensor: torch.Size([64, 3, 3, 3])
Print the Dims we want to keep: [0, 1]
norm shape = torch.Size([64, 3])
Number Of Features Map in current  layer l     = 64
Number Of Features Map in previous layer (l-1) = 3
...
Shape of the tensor: torch.Size([64, 64, 3, 3])
Print the Dims we want to keep: [0, 1]
norm shape = torch.Size([64, 64])
Number Of Features Map in current  layer l     = 64
Number Of Features Map in previous layer (l-1) = 64
................................................................
Shape of the tensor: torch.Size([128, 64, 3, 3])
Print the Dims we want to keep: [0, 1]
norm shape = torch.Size([128, 64])
Number Of Features Map in current  layer l     = 128
Number Of Features Map in previous layer (l-1) = 64
................................................................
Shape of the tensor: torch.Size([128, 128, 3, 3])
Print the Dims we want to keep: [0, 1]
norm shape = torch.Size([128, 128])
Number Of Features Map in current  layer l     = 128
Number Of Fe

#### *Sort the list of score in the ascending order*

In [5]:
for i in range(len(candidateConvLayer)):
    fp.sort_kernel_by_distance(candidateConvLayer[i])

In [6]:
print(f'c={len(candidateConvLayer)}, c[] ={len(candidateConvLayer[1])}, c[][] = {len(candidateConvLayer[1][0])},c[][][] = {len(candidateConvLayer[1][0][0])}')

c=4, c[] =64, c[][] = 176,c[][][] = 4


#### *Extract K Elements of ascending List of Score and they will be the minimum score*

In [8]:
prune_count = ip.getPruneCount(module=module,blocks=blockList,maxpr=0.1,)
newList = []
for i in range(len(candidateConvLayer)):
    newList.append(fp.get_k_element(channel_list=candidateConvLayer[i],k=prune_count[i])  )
print(f'c={len(newList)}, c[] ={len(newList[1])},' 
      f'c[][] = {len(newList[1][0])},c[][][] = {len(newList[1][0][0])}')

c=4, c[] =64,c[][] = 1,c[][][] = 4


### Now Execute the pruning

#### Define class to execute custom pruning

In [9]:
import torch.nn.utils.prune as prune
class KernalPruningMethod(prune.BasePruningMethod):
    PRUNING_TYPE = 'unstructured'
    def compute_mask(self, t, default_mask):
        mask = default_mask.clone()
        #mask.view(-1)[::2] = 0
        size = t.shape
        print(size)
        for k1 in range(len(newList[layer_number])):
            for k2 in range(len(newList[layer_number][k1])):
                i= newList[layer_number][k1][k2][1]
                j= newList[layer_number][k1][k2][0]
                
                #print(f"i= {i} , j= {j}")
                
                mask[i][j] = 0
        return mask
def kernal_unstructured(module, name):
    KernalPruningMethod.apply(module, name)
    return module

### *Pass the cadidate layer of Conv Net for custom pruning*

In [10]:
layer_number = 0
for i in range(en):
    layer_number = i
    kernal_unstructured(module[i], name = 'weight')
    layer_number += 1

torch.Size([64, 3, 3, 3])
torch.Size([64, 64, 3, 3])
torch.Size([128, 64, 3, 3])
torch.Size([128, 128, 3, 3])


##### commit the pruning parameter 

In [11]:
for i in range(en):
    prune.remove(module[i], 'weight')

### Deep Copy Code

In [23]:
j=0
i=0
while j < en:
    if featureList[i] == 'M':
        i+=1
        continue
    else:
        featureList[i] -= prune_count[j]
        j+=1
        i+=1


In [24]:
print(featureList)

[63, 63, 'M', 125, 125, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M']


In [22]:
featureList = ip.createFeatureList(newModel)

In [26]:
tempModel = lm.create_vgg_from_feature_list(featureList)
print(tempModel)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 63, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(63, 63, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(63, 125, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(125, 125, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(125, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (15): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (16): 